In [1]:
%pip install onnxruntime

%pip install -U scikit-image

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U git+https://github.com/albumentations-team/albumentations

  Cloning https://github.com/albumentations-team/albumentations to c:\users\feers\appdata\local\temp\pip-req-build-d14quvu4
  Resolved https://github.com/albumentations-team/albumentations to commit f5ae3f5a9d8b77182c8a726deeadfc5dd1e93b8c
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/albumentations-team/albumentations 'C:\Users\feers\AppData\Local\Temp\pip-req-build-d14quvu4'


In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [1]:
import os
import random
import shutil
import albumentations as A
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from concurrent.futures import ThreadPoolExecutor, as_completed

def image_processing(img, threshold_value=127, kernel_size=3, dilate_iterations=1):
    """
    Proses gambar menggunakan grayscale thresholding dan dilasi.
    
    Parameters:
        img (numpy.ndarray): Input gambar.
        threshold_value (int): Nilai threshold (0-255).
        kernel_size (int): Ukuran kernel untuk dilasi.
        dilate_iterations (int): Jumlah iterasi untuk dilasi.
    
    Returns:
        numpy.ndarray: Gambar hasil thresholding dan dilasi.
    """
    # Konversi gambar ke grayscale
    if len(img.shape) == 3:  # Jika memiliki 3 channel (RGB/BGR)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray_img = img

    # Thresholding
    _, binary_img = cv2.threshold(gray_img, threshold_value, 255, cv2.THRESH_BINARY)

    # Kernel untuk dilasi
    kernel = np.ones((kernel_size, kernel_size), np.uint8)

    # Dilasi
    dilated_img = cv2.dilate(binary_img, kernel, iterations=dilate_iterations)

    return dilated_img

# Path ke folder data
images_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\EMDS7"
labels_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\EMDS7xml"
output_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\\temp"
sample_output_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\sample_output"

train_val_split = 0.8  # Rasio data train/validation

# Cek apakah path gambar dan label ada
if not os.path.exists(images_dir):
    raise FileNotFoundError(f"Path gambar tidak ditemukan: {images_dir}")
if not os.path.exists(labels_dir):
    raise FileNotFoundError(f"Path label tidak ditemukan: {labels_dir}")

# Remove output directory if it exists and create fresh output folders
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

if os.path.exists(sample_output_dir):
    shutil.rmtree(sample_output_dir)

# Buat ulang folder output yang fresh
os.makedirs(os.path.join(output_dir, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels/val'), exist_ok=True)
os.makedirs(sample_output_dir, exist_ok=True)

# Daftar semua gambar dan annotations
image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]
label_files = [f.replace('.png', '.xml') for f in image_files]

# Shuffle data for randomness
data = list(zip(image_files, label_files))
random.shuffle(data)

# Split data into training and validation sets
split_index = int(train_val_split * len(data))
train_data = data[:split_index]
val_data = data[split_index:]

class_mapping = {
    'G001': 0,  'G002': 1,  'G003': 2,  'G004': 3,  'G005': 4,  'G006': 5,  'G007': 6,  'G008': 7,  'G009': 8,  'G010': 9,
    'G011': 10, 'G012': 11, 'G013': 12, 'G014': 13, 'G015': 14, 'G016': 15, 'G017': 16, 'G018': 17, 'G019': 18, 'G020': 19,
    'G021': 20, 'G022': 21, 'G023': 22, 'G024': 23, 'G025': 24, 'G026': 25, 'G027': 26, 'G028': 27, 'G029': 28, 'G030': 29,
    'G031': 30, 'G032': 31, 'G033': 32, 'G034': 33, 'G035': 34, 'G036': 35, 'G037': 36, 'G038': 37, 'G039': 38, 'G040': 39,
    'G041': 40, 'G042': 41, 'G043': 42, 'G044': 43, 'G045': 44, 'G046': 45, 'G047': 46, 'G048': 47, 'G049': 48, 'G050': 49,
    'G051': 50, 'G052': 51, 'G053': 52
}

# Augmentasi gambar dan bounding box tanpa mirroring, mengisi area kosong dengan putih
augmentor = A.Compose([
    A.Resize(1024, 1024, p=0)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

def draw_bboxes(image, bboxes, class_labels):
    """Gambar bounding box pada gambar."""
    for bbox, class_id in zip(bboxes, class_labels):
        x_center, y_center, width, height = bbox
        img_h, img_w = image.shape[:2]
        xmin = int((x_center - width / 2) * img_w)
        ymin = int((y_center - height / 2) * img_h)
        xmax = int((x_center + width / 2) * img_w)
        ymax = int((y_center + height / 2) * img_h)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv2.putText(image, str(class_id), (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)

def convert_xml_to_txt(xml_path, txt_output_path, augment=False, img=None):
    """Fungsi ini akan mengonversi format XML ke format YOLO (txt)."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)
    bboxes = []
    class_labels = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        class_id = class_mapping.get(class_name, -1)
        if class_id == -1:
            continue
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        x_center = (xmin + xmax) / 2.0 / width
        y_center = (ymin + ymax) / 2.0 / height
        box_width = (xmax - xmin) / width
        box_height = (ymax - ymin) / height
        bboxes.append([x_center, y_center, box_width, box_height])
        class_labels.append(class_id)

    # Pastikan setidaknya ada satu kelas yang valid
    if not class_labels:
        return None, None, None

    if augment and img is not None:
        augmented = augmentor(image=img, bboxes=bboxes, class_labels=class_labels)
        img = augmented['image']
        bboxes = augmented['bboxes']
        class_labels = augmented['class_labels']
    
    with open(txt_output_path, 'w') as txt_file:
        for bbox, label in zip(bboxes, class_labels):
            txt_file.write(f"{label} {' '.join(map(str, bbox))}\n")

    return img, bboxes, class_labels

def process_data(img_file, lbl_file, img_dest_folder, lbl_dest_folder, sample_output_count=15, sample_counter=0):
    img_src_path = os.path.join(images_dir, img_file)
    lbl_src_path = os.path.join(labels_dir, lbl_file)
    img = cv2.imread(img_src_path)
    
    # Terapkan pemrosesan gambar
    processed_img = image_processing(img)
    
    img_dest_path = os.path.join(img_dest_folder, img_file)
    lbl_dest_path = os.path.join(lbl_dest_folder, lbl_file.replace('.xml', '.txt'))
    
    # Konversi XML ke TXT dan simpan gambar yang diproses
    processed_img, bboxes, class_labels = convert_xml_to_txt(lbl_src_path, lbl_dest_path, img=processed_img)
    
    # Jika tidak ada kelas yang valid, lewati gambar ini
    if processed_img is None:
        print(f"Skipping image {img_file} due to no valid classes.")
        return
    
    # Simpan gambar yang diproses
    cv2.imwrite(img_dest_path, processed_img)
    
    # Simpan contoh gambar dengan bounding box jika diperlukan
    if sample_counter < sample_output_count:
        sample_img = processed_img.copy()
        draw_bboxes(sample_img, bboxes, class_labels)
        cv2.imwrite(os.path.join(sample_output_dir, f"{img_file.replace('.png', '_sample.png')}"), sample_img)
        sample_counter += 1
    
    print(f"Processed Image saved to: {img_dest_path}")

def copy_and_convert_data(data, img_dest_folder, lbl_dest_folder, sample_output_count=30):
    sample_counter = 0
    with ThreadPoolExecutor() as executor:
        futures = []
        for img_file, lbl_file in data:
            futures.append(executor.submit(process_data, img_file, lbl_file, img_dest_folder, lbl_dest_folder, sample_output_count, sample_counter))
        for future in as_completed(futures):
            future.result()

copy_and_convert_data(train_data, os.path.join(output_dir, 'images/train'), os.path.join(output_dir, 'labels/train'))
copy_and_convert_data(val_data, os.path.join(output_dir, 'images/val'), os.path.join(output_dir, 'labels/val'))

print("Data berhasil diproses dan dikonversi ke format YOLO! Contoh gambar disimpan di folder sample_output.")

Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G017-067-0400.png
Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G032-027-0400.png
Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G038-017-0400.png
Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\images/train\EMDS7-G035-049-0400.png
Processed Image saved to: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\im

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [3]:
from ultralytics import YOLO

name = 'it2'
def main():
    # Inisialisasi model YOLOv8
    model = YOLO('yolov8m.pt')  # Pre-trained model weights untuk YOLOv8
    
    # Tambahkan `class_weights` sebagai parameter dinamis
    model.train(
        data='../data.yaml',
        epochs=100,
        imgsz=640,
        batch=16,
        name=name,
        workers=16,
        project=f'runs/train/{name}',
        device=0,
        augment=False
    )

    # Evaluasi model setelah pelatihan
    model.val()

    # Export model ke format lain (misal: TorchScript, ONNX, CoreML, TensorRT)
    model.export(format='onnx')  # atau format lain yang diinginkan

if __name__ == '__main__':
    main()

WARNING  Python>=3.10 is required, but Python==3.9.20 is currently installed 
New https://pypi.org/project/ultralytics/8.3.38 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.21  Python-3.9.20 torch-2.5.0 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=../data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=16, project=runs/train/it2, name=it236, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, a

train: Scanning D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\train... 1872 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1872/1872 [00:04<00:00, 465.25it/s]

train: New cache created: D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\train.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\val... 467 images, 0 backgrounds, 0 corrupt: 100%|██████████| 467/467 [00:01<00:00, 402.42it/s]

...


100 epochs completed in 1.799 hours.
Optimizer stripped from runs\train\it2\it236\weights\last.pt, 52.1MB
Optimizer stripped from runs\train\it2\it236\weights\best.pt, 52.1MB

Validating runs\train\it2\it236\weights\best.pt...
Ultralytics 8.3.21  Python-3.9.20 torch-2.5.0 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,863,499 parameters, 0 gradients, 78.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:04<00:00,  3.41it/s]


                   all        467       1106      0.865      0.733      0.785      0.666
          Oscillatoria         15         46       0.75      0.478       0.53      0.401
        Ankistrodesmus         14         14      0.332      0.143      0.193      0.169
           Microcystis         96        174       0.75      0.672      0.661      0.516
            Gomphonema         17         17          1      0.912      0.992      0.885
         Sphaerocystis          9          9          1      0.947      0.995      0.831
             Cosmarium          3          3      0.756      0.333       0.41      0.283
             Cocconeis          1          1      0.782          1      0.995      0.895
             Tribonema         16         20      0.907      0.487      0.593      0.551
             Chlorella         40         43        0.9      0.628      0.708      0.575
            Tetraedron          4         10      0.949          1      0.995      0.889
        Ankistrodesmu

val: Scanning D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\temp\labels\val.cache... 467 images, 0 backgrounds, 0 corrupt: 100%|██████████| 467/467 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:09<00:00,  3.31it/s]


                   all        467       1106      0.865      0.733      0.785      0.665
          Oscillatoria         15         46      0.749      0.478      0.528      0.396
        Ankistrodesmus         14         14      0.333      0.143      0.193      0.169
           Microcystis         96        174       0.75      0.672      0.661      0.516
            Gomphonema         17         17          1      0.912      0.992      0.879
         Sphaerocystis          9          9          1      0.947      0.995      0.827
             Cosmarium          3          3      0.755      0.333       0.41      0.283
             Cocconeis          1          1      0.782          1      0.995      0.895
             Tribonema         16         20      0.907      0.488      0.595      0.549
             Chlorella         40         43      0.901      0.628      0.709      0.571
            Tetraedron          4         10      0.949          1      0.995      0.879
        Ankistrodesmu

In [ ]:
import shutil
import os
import datetime

source_dir = os.getcwd()
destination_dir = f'runs/train/{name}'

os.makedirs(destination_dir, exist_ok=True)

for filename in os.listdir(source_dir):
    file_path = os.path.join(source_dir, filename)
    if os.path.isfile(file_path):
        shutil.move(file_path, destination_dir)

True
